In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Identifying the Duplicate Questions


In [ ]:
## Importing the suitable libraries
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
stp= stopwords.words()
from nltk.stem import WordNetLemmatizer
from matplotlib import style
from collections import Counter
style.use("fivethirtyeight")
%matplotlib inline
lemm= WordNetLemmatizer()
from keras import layers
from keras import Model
from keras.utils import np_utils
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Input
from keras.utils import to_categorical
import string
import warnings
warnings.filterwarnings("ignore")


## Reading the data

In [ ]:
## Importing the libraries
data= pd.read_csv("../input/train.csv")
data_t= pd.read_csv("../input/test.csv")

In [ ]:
## Printing the data
data.head()

In [ ]:
def get_unigrams(text):
    return([word.lower() for word in text.split() if w not in stp])
def count_common(text):
    return(len(set(text["question1_m"]).intersection(set(text["question2_m"]))))
def get_common_unigram_ratio(row):
    return float(row["common_w"]) / max(len( set(row["question1_m"]).union(set(row["question2_m"]))),1)

In [ ]:
data["common_w"]= data.apply(lambda x: count_common(x),axis=1)
data["common_ratio"]= data.apply(lambda x: get_common_unigram_ratio(x),axis=1)

In [ ]:
sns.distplot(data["common_ratio"], kde=True)

In [ ]:
wc= data["common_w"].value_counts()
sns.barplot(wc.index, wc.values)

In [ ]:
## Function to filter out the stopwords and irrelvent punctuations
def filter_q(sent):
    table= str.maketrans("","",string.punctuation)
    strr= [w.translate(table) for w in str(sent).split()]
    strr= [w.lower() for w in strr]
    strr= [lemm.lemmatize(w) for w in strr if w not in stp]
    strr= [w for w in strr if w.isalpha()]
    strr= [w for w in strr if len(w)>2]
    t_sent= " ".join(strr)
    return(t_sent)

In [ ]:
data["question1_m"]= data["question1"].apply(filter_q)

In [ ]:
data["question2_m"]= data["question2"].apply(filter_q)

In [ ]:
token1= Tokenizer()
token2= Tokenizer()
token1.fit_on_texts(data["question1_m"])
token2.fit_on_texts(data["question2_m"])

In [ ]:
total_w1= len(token1.word_index)+1
total_w2= len(token2.word_index)+1
max_q1= max([len(i) for i in quest_1])
max_q2= max([len(j) for j in quest_2])

In [ ]:
quest_1= token1.texts_to_sequences(data["question1_m"])
quest_2= token2.texts_to_sequences(data["question2_m"])
quest_1= pad_sequences(quest_1, maxlen= max_q1)
quest_2= pad_sequences(quest_2, maxlen= max_q2)

## Base model :

In [ ]:
# Question1
input_1= Input(shape=(None,), dtype="int32",name="ques1")
embedd_text= layers.Embedding(total_w1,64)(input_1)
lstm_l1= layers.LSTM(62,recurrent_dropout=0.2)(embedd_text)
## Question2
input_2= Input(shape=(None,), dtype="int32",name="ques2")
embedd_text= layers.Embedding(total_w2,96)(input_2)
lstm_l2= layers.LSTM(62, recurrent_dropout=0.2)(embedd_text)
concat= layers.concatenate([lstm_l1,lstm_l2],axis=1)
answer= layers.Dense(2,activation="softmax")(concat)
model = Model([input_1, input_2], answer)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

In [ ]:
output =np_utils.to_categorical(data["is_duplicate"])

In [ ]:
model.fit([quest_1,quest_2],output,epochs=2,batch_size=128,verbose=1)

In [ ]:
sns.countplot(data["is_duplicate"])

In [ ]:
Counter(data["qid1"])

In [ ]:
ser= pd.Series(data["question1"].tolist() + data["question2"].tolist()).astype(str)

In [ ]:
count_trqser = ser.apply(len)

In [ ]:
count_trqser.plot("hist")

In [ ]:
ser1= data["question1_m"].astype("str")
ser2= data["question2_m"].astype("str")

In [ ]:
serw_1=  " ".join(ser1)
serw_2= " ".join(ser2)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
w_c= WordCloud(stopwords= STOPWORDS, background_color="white", height= 2000, width =4000).generate(serw_1)
plt.imshow(w_c),
plt.figure(figsize=(16,8))
plt.axis("off")
plt.show()


In [ ]:
wc_2= WordCloud(stopwords= STOPWORDS, background_color="white", height= 2000, width=4000).generate(serw_2)
plt.imshow(wc_2)
plt.figure(figsize=(16,8))
plt.axis("off")
plt.show()

In [ ]:
sns.boxplot(data["is_duplicate"],data["common_ratio"])

In [ ]:
sns.boxplot(data["is_duplicate"],data["common_w"])

* Thus from the above two plot we can estimate that common ratio is the best estimator to differentiate between duplicate and non duplicate question

In [ ]:
from collections import defaultdict
q_dict= defaultdict(set)
ques= pd.concat([data[["question1","question2"]],data_t[["question1","question2"]]], axis=0)

In [ ]:
ques

In [ ]:
from collections import defaultdict
q_dict= defaultdict(set)
for i in range(ques.shape[0]):
    q_dict[ques.question1[i].add(ques.question2[i])]
    

In [ ]:
q_dict["a"].add("b")

In [ ]:
def find_l(a):
    return(len(str(a).split()))

In [ ]:
ques["q1_l"]= ques["question1"].apply(find_l)

In [ ]:
data["ques1_len"] = data["question1"].apply(find_l)

In [ ]:
data["ques2_len"]= data["question2"].apply(find_l)

In [ ]:
max(data["ques2_len"])

In [ ]:
pvt_df= data.pivot_table(index= "ques1_len", columns="ques2_len", values= "is_duplicate")
sns.heatmap(pvt_df)
plt.title("Mean is_duplicate value distribution acrosss q1 and q2 frequency")
plt.figure(figsize=(12,12))
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
val = ques["q1_l"].value_counts()
sns.barplot(val.index,val.values, alpha=0.5)
plt.xticks(rotation="vertical")

In [ ]:
plt.figure(figsize=(12,18))
data_glf= data.groupby("ques1_len")["is_duplicate"].agg("mean")
sns.barplot(data["ques1_len"].value_counts().values, data_glf.values)
plt.xticks(rotation="vertical")

In [ ]:
# Question1
input_1= Input(shape=(None,), dtype="int32",name="ques1")
embedd_text= layers.Embedding(total_w1,64)(input_1)
lstm_l1= layers.LSTM(62,recurrent_dropout=0.2)(embedd_text)
## Question2
input_2= Input(shape=(None,), dtype="int32",name="ques2")
embedd_text= layers.Embedding(total_w2,96)(input_2)
lstm_l2= layers.LSTM(62, recurrent_dropout=0.2)(embedd_text)

input_3= Input(shape=(None,),dtype="int32",name="common_w")
den_2= layers.Dense(2,activation="softmax")(input_3)

concat= layers.concatenate([lstm_l1,lstm_l2,den_2],axis=1)
answer= layers.Dense(2,activation="softmax")(concat)
model = Model([input_1, input_2, input_3], answer)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

In [ ]:
model.fit([quest_1,quest_2,data["common_ratio"]],output,epochs=2,batch_size=128,verbose=1)